In [18]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [19]:
precision_matrices = torch.load("../checkpoints/precision_matrices_hq.pt")

fairseq_ckpt = torch.load("../checkpoints/checkpoint_best.pt")
fairseq_model_dict = dict(fairseq_ckpt["model"])

In [20]:
total, k = 0, 5
masks = {}

for n in precision_matrices.keys():
    if fairseq_model_dict[n].dim() == 2 and ("embed" not in n):
        p = precision_matrices[n]
        u, s = torch.mean(p), torch.std(p)
        p_msk = (p > (u + k*s))
        total += p_msk.sum()
        masks[n] = p_msk
        
print("total", total)

torch.save(masks, "../checkpoints/masks.pt")

total tensor(124545, device='cuda:0')


In [21]:
# for n in precision_matrices.keys():
#     if fairseq_model_dict[n].dim() == 2:
#         fig, ax = plt.subplots(2, 3, figsize=(10, 10))
#         p1_og = precision_matrices[n].cpu().numpy()

#         try:
#             p1 = np.log(precision_matrices[n].cpu().numpy())
#             u1, s1, mx1, mn1 = p1.mean(), p1.std(), p1.max(), p1.min()
            
#             pixel_plot1 = ax[0, 0].imshow(
#                 p1, 
#                 cmap="hot",
#                 origin='lower'
#             )
#             plt.colorbar(pixel_plot1, orientation=('vertical' if (n.endswith("fc1.weight") or n.endswith("embed_tokens.weight")) else 'horizontal'), ax=ax[0, 0])
#             ax[0, 0].set_title(f"{n}\n\nMean: {u1}\nStd: {s1}\nMax: {mx1}\nMin: {mn1}", fontsize=10)

#             p1_flat = p1.flatten()
#             ax[1, 0].hist(p1_flat, bins="auto")
#             ax[1, 0].axvline(p1_flat.mean(), color='r', linestyle="--")
#             ax[1, 0].set_xlabel("log(p)")
#             ax[1, 0].set_title(f"Distribution of parameters in precision-matrix of\n{n}", fontsize=10)
#         except ValueError:
#             pass
        
#         ###########################################################################################################################

#         p2 = fairseq_model_dict[n].cpu().numpy()
#         u2, s2, mx2, mn2 = p2.mean(), p2.std(), p2.max(), p2.min()
#         pixel_plot2 = ax[0, 1].imshow(
#             p2, 
#             cmap="hot",
#             origin='lower'
#         )
#         plt.colorbar(pixel_plot2, orientation=('vertical' if (n.endswith("fc1.weight") or n.endswith("embed_tokens.weight")) else 'horizontal'), ax=ax[0, 1])
#         ax[0, 1].set_title(f"{n}\n\nMean: {u2}\nStd: {s2}\nMax: {mx2}\nMin: {mn2}", fontsize=10)

#         p2_flat = p2.flatten()
#         ax[1, 1].hist(p2_flat, bins="auto")
#         ax[1, 1].axvline(p2_flat.mean(), color='r', linestyle="--")
#         ax[1, 1].set_title(f"Distribution of parameters in\n{n}", fontsize=10)
    
#         ###########################################################################################################################

#         p2_p1 = p1_og * p2
#         pixel_plot3 = ax[0, 2].imshow(
#             p2_p1, 
#             cmap="hot",
#             origin='lower'
#         )
#         plt.colorbar(pixel_plot3, orientation=('vertical' if (n.endswith("fc1.weight") or n.endswith("embed_tokens.weight")) else 'horizontal'), ax=ax[0, 2])

#         p2_p1_flat = p2_p1.flatten()
#         ax[1, 2].hist(p2_flat, bins="auto")
#         ax[1, 2].axvline(p2_p1_flat.mean(), color='r', linestyle="--")


#         # msk_lo = p1 < u1
#         # msk_mid = (p1 >= u1) & (p1 <= (u1 + 10*s1))
#         # msk_hi = p1 > (u1 + 10*s1)

#         # p2[msk_lo] = -1
#         # p2[msk_mid] = 0
#         # p2[msk_hi] = 1

#         # pixel_plot3 = ax3.imshow(
#         #     p2, 
#         #     cmap="hot",
#         #     origin='lower'
#         # )
#         # plt.colorbar(pixel_plot3, orientation=('vertical' if n.endswith("fc1.weight") else 'horizontal'), ax=ax3)
#         # ax3.set_title(f"Parameter importance", fontsize=10)
        
#         plt.tight_layout()
#         plt.show()